In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from torch.utils.data import Dataset, SubsetRandomSampler, DataLoader

In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)  
a = torch.zeros(4,3)    
a = a.to(device)

In [3]:
labels = {}
#W, R, 1, 2, 3, 4, M 
labels['W']=5
labels['R']=6
labels['M']=0
labels['1']=1
labels['2']=2
labels['3']=3
labels['4']=4

In [29]:
def load_data(dataset):
    batch_size=32
    validation_split = .2
    shuffle_dataset = True
    random_seed= 42
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    dataset.x = dataset.x.to(device)
    dataset.y = dataset.y.to(device)
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=valid_sampler)


    return train_loader, val_loader

In [30]:

class CustomSleepDataset(Dataset):
    
    def __init__(self, file_list):
        self.x = []
        self.y = []
        for i in file_list.keys():
            self.parse(file_list[i]["PSG.edf"], file_list[i]["Hypnogram.edf"])
        self.x = torch.FloatTensor(self.x)
        self.y = torch.LongTensor(self.y)
    def parse(self, sleep, hypnogram):
        _,_, header = highlevel.read_edf(hypnogram)
        signals, _, _ = highlevel.read_edf(sleep)
     
        for annotation in header['annotations']:
            start = int(annotation[0])
            end = int(annotation[1])
            sleep_stage = annotation[2][-1]
            if sleep_stage != '?':
                for i in range(start, end, 30):
                # pick i to i +30 for target range end
                    self.x.append([ signals[0][i*100:(i+30)*100],signals[1][i*100:(i+30)*100], signals[2][i*100:(i+30)*100]])
                    self.y.append(labels[sleep_stage])
    
    def __len__(self):

        return len(self.y)

    
    def __getitem__(self, index):
        
        return (self.x[index], self.y[index])
        


In [31]:
filename = 'sleeper_dataset.pkl'
infile = open(filename,'rb')
dataset = pickle.load(infile)
infile.close()
train_loader, val_loader = load_data(dataset)

In [32]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=256, kernel_size=101,stride=10,padding=51)
        #kernel =2, stride=2
        self.pool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=11,stride=2,padding=6)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=11,stride=2,padding=6)
        
        self.fc1 = nn.Linear(64 * 10, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
#         print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
#         print(x.shape)
        x = self.pool(F.relu(self.conv3(x)))
#         print(x.shape)
        x = x.view(-1,64 * 10)
        x = self.fc1(x)
        return x
    

In [33]:
##load model
model = SimpleCNN()

In [34]:
criterion = torch.nn.modules.loss.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [35]:
##train
n_epochs = 10

def train_model(model, train_dataloader, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim
    model.train()# prep model for training
    model.to(device)
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        for data, target in train_dataloader:
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(data)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, target)
                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()
           
            curr_epoch_loss.append(loss.cpu().data.numpy())
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")
    return model


In [36]:

train_model(model, train_loader,n_epochs, optimizer, criterion)

Epoch 0: curr_epoch_loss=0.0014718712773174047
Epoch 1: curr_epoch_loss=0.0
Epoch 2: curr_epoch_loss=0.0
Epoch 3: curr_epoch_loss=0.0
Epoch 4: curr_epoch_loss=0.0
Epoch 5: curr_epoch_loss=0.0
Epoch 6: curr_epoch_loss=0.0
Epoch 7: curr_epoch_loss=0.0
Epoch 8: curr_epoch_loss=0.0
Epoch 9: curr_epoch_loss=0.0


SimpleCNN(
  (conv1): Conv1d(3, 256, kernel_size=(201,), stride=(10,), padding=(51,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(256, 128, kernel_size=(11,), stride=(2,), padding=(6,))
  (conv3): Conv1d(128, 64, kernel_size=(11,), stride=(2,), padding=(6,))
  (fc1): Linear(in_features=640, out_features=7, bias=True)
)

In [37]:
def eval_model(model, dataloader):

    model.eval()
    Y_pred = []
    Y_test = []
    for data, target in dataloader:
        outputs = model(data)
        _,outputs = torch.max(outputs, dim = 1)
        Y_pred.append(outputs)

        Y_test.append(target.detach().numpy())
    Y_pred = np.concatenate(Y_pred, axis=0)
    Y_test = np.concatenate(Y_test, axis=0)

    return Y_pred, Y_test

In [38]:
from sklearn.metrics import accuracy_score

y_pred, y_true = eval_model(model, val_loader)
acc = accuracy_score(y_true, y_pred)

ModuleNotFoundError: No module named 'sklearn'